Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Using Google Search as a grounding service.


# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.2 MB/s eta 0:00:00


## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [3]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [4]:
import vertexai
from vertexai.preview.generative_models import grounding
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
)

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Prompt sent to LLM

In [18]:
# PROMPT = (
#     "What is the latest news related to the key interest rate announced by FED after mid of May 2024"
# )

PROMPT = (
    "Could you recommend a movie related to current Korean politic situations"
)

# PROMPT = (
#     "현재 한국의 정치 상황에 맞는 예전에 상영된 영화 제목을 알려주세요."
# )

## Response without grounding services.
This response is information from LLM without grounding services to enrich contexts sent to LLM.

In [19]:
response = model.generate_content(PROMPT)
display(Markdown(response.text))

It's tricky to recommend a movie directly reflecting the *current* political situation in South Korea. Politics are constantly evolving, and movies generally take time to develop and release, making it hard to find something truly up-to-the-minute. 

However, I can recommend some films that explore themes relevant to Korean politics and societal issues that often influence political discourse:

**For a general understanding of Korean society and its complexities:**

* **"Parasite" (2019):** This Oscar-winning film is a powerful allegory for class inequality and social mobility in South Korea. While not directly political, it sheds light on the societal tensions that often fuel political debates.
* **"Burning" (2018):** A haunting thriller that examines themes of social alienation, class disparity, and the struggles of young people in a rapidly changing Korea.

**For a closer look at specific historical events or political figures:**

* **"The Handmaiden" (2016):** Based on a novel, this erotic thriller set during the Japanese occupation of Korea explores themes of power, manipulation, and the complexities of identity.
* **"1987: When the Day Comes" (2017):** A powerful historical drama about the 1987 pro-democracy protests, highlighting the struggles for democracy and human rights in Korea.
* **"The Age of Shadows" (2016):** A historical action thriller set during the Japanese occupation of Korea, focusing on the fight for independence and the complexities of resistance.

**For a more contemporary perspective:**

* **"Train to Busan" (2016):** While a zombie movie, it features themes of social inequality and government incompetence, reflecting some anxieties about Korean society.
* **"The King" (2017):** A political thriller about a prosecutor who rises to power through ambition and ruthless tactics, offering a critical look at corruption and power dynamics within Korean politics.
* **"The Spy Gone North" (2018):** Based on a true story, this film focuses on a North Korean spy who infiltrates South Korea, offering a glimpse into the ongoing tensions and ideological differences between the two Koreas.

Remember, these are just starting points. It's always good to research further based on your specific interests and desired depth of exploration. You can find many documentaries, dramas, and films on streaming platforms that cover diverse perspectives on Korean politics and society. 


##Configuration Google Search

In [20]:
def get_response(prompt:str)->str:

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  response = model.generate_content(
      prompt,
      tools=[tool],
      generation_config=GenerationConfig(
          temperature=0.0,
      ),
  )

  return response.text


In [21]:
outcome = get_response(PROMPT)
display(Markdown(outcome))

It's difficult to recommend a specific movie that perfectly reflects the current Korean political situation, as movies often focus on historical events or fictionalized scenarios. However, based on your provided sources, here are some movies that explore themes relevant to Korean politics and might offer insights into the country's political landscape:

* **"1987: When the Day Comes" (2017):** This critically acclaimed film depicts the events leading up to and surrounding the death of a student protester, sparking the June Democratic Uprising, which significantly impacted South Korea's political trajectory. It provides a glimpse into the country's struggle for democracy and the impact of political activism.
* **"A Taxi Driver" (2017):** This film focuses on the Gwangju Democratization Movement of May 1980, a pivotal event in South Korean history. It explores the government's response to protests and the role of ordinary citizens in challenging authority.
* **"The President's Last Bang" (2005):** This film offers a fictionalized account of the assassination of Park Chung-hee, a controversial figure in South Korean history. It explores the complexities of his legacy, highlighting both his contributions to economic development and his authoritarian rule.
* **"The President's Barber" (2004):** This political satire follows a barber who becomes entangled in the inner circle of the South Korean dictator Park Chung-hee. It provides a satirical perspective on the political landscape and the consequences of power.

These films offer a range of perspectives on Korean politics, from historical events to fictionalized narratives. While they may not directly reflect the current political situation, they provide valuable insights into the country's political history, societal dynamics, and the complexities of power. 
